In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm, gamma, lognorm, weibull_min, expon
plt.rc('legend', fontsize=14) 

In [ ]:
colors=[[0,0.5,0.5],
        [1.0000,0.6666,0],
        [0.8000,0.1111,0],
        [0.500,0.500,0.0],
        [0.0500,0.0500,0.0500],
        [0.0000,1.0000,0.3333],
        [0.800,1,0.0],
        [0.3,0.3,0.3],
        [0.8,0.8,0.8]]

# Load Data

In [ ]:
# Load Data
#df = pd.read_csv('LEOSS Kuehn 08 25/SUF_Kuehn_200825.csv', sep=';')
df = pd.read_csv('LEOSS_Kuehn_2020_10_14/SUF_Kuehn_201014.csv', sep=';')

# Only keep entries that either recovered or died specifically from Covid-19
df = df[np.any([(df['BL_LastKnownStatus'] == 2).values, (df['BL_LastKnownStatus'] == 3).values], axis=0)]

# Discard entries with unknown age and convert to int
df = df[df['BL_Age']!='*']
df['BL_Age'] = df['BL_Age'].astype(int)


In [ ]:
# Reads the values of a single column, which consits of strings. 
# The entry of the column may contain values like '<3' and '>15',
# which are replaced by lower_bound or upper_bound
def read_column(df, lower_bound=None, upper_bound=None):
    duration = []
    for i in range(len(df)):
        if ';' in df.values[i]:
            # outliers are grouped for anonymization, there is a ";" in the string and
            # they are then given as lower_bound;upper_bound then
            sc_ind = df.values[i].find(';') # find the index of the ";" in the string
            
            lower_bound =int(df.values[i][1:sc_ind]) # get lower bound
            upper_bound =int(df.values[i][sc_ind+1:-1]) # get upper bound
            
            # set new value as mean
            if int((lower_bound + upper_bound)/2) >= 0:
                duration.append(int((lower_bound + upper_bound)/2))
            
        # some values are only provided as ">n" oder "<n"
        # then, set the actual value to n+1 or n-1
        elif '>' in df.values[i]:
            duration.append(int(df.values[i][2:])+1)
        elif '<' in df.values[i]:
            if int(df.values[i][2:])-1 >= 0:
                duration.append(int(df.values[i][2:])-1)
        else:
            if(int(df.values[i])>= 0):
                duration.append(int(df.values[i]))
            
    print("Number of Patients: ", len(duration))
            
    return duration

In [ ]:
# save a dataframe for an age division to a csv file
def save_df(ages, unique_list, count_list, name):
    min_u = 0
    max_u = -100
    for age in range(len(ages)):
        if len(unique_list[age])>0:
            min_u = min(min_u, min(unique_list[age]))
            max_u = max(max_u, max(unique_list[age]))

    cols = [str(x) for x in range(min_u, max_u + 1)]
    count_data = np.zeros((len(ages), len(cols)))
    for age in range(len(ages)):
        counter = 0
        if len(unique_list[age])>0:
            for i in range(len(cols)):
                if int(cols[i]) == unique_list[age][counter]:
                    count_data[age, i] = count_list[age][counter]
                    if counter < len(unique_list[age])-1:
                        counter += 1
                
    count_df = pd.DataFrame(count_data, columns=cols, index=ages)
    display(count_df)
    count_df.to_csv(name + '.csv')

In [ ]:
# Calculates probabilities of ICU transfer during hospital stay (mu_HU)
# and dying during ICU (mu_UD)
# also computes confidence intervals based on Walds method for the Binomial distribution
# https://en.wikipedia.org/wiki/Binomial_distribution
z = 1.96 # ensures 95% confidence interval

def calc_prob(df_age_groups, ages):
    mu_HU_list = []
    mu_UD_list = []
    mu_HU_dev_list = []
    mu_UD_dev_list = []    
    for (df_age, age) in zip(df_age_groups, ages):
        df_hosp = df_age[['BL_Duration_InpatientstayNewCat', 'BL_Duration_ICUStayNewCat',
                          'BL_LastKnownStatus', 'BL_AdmissionComputedNewCat',
                          'BL_ICU', 'BL_Inpatient']]
        df_hosp = df_hosp[df_hosp['BL_Inpatient']=='1']
        df_icu = df_hosp[df_hosp['BL_ICU']=='1']
        df_dead = df_icu[df_icu['BL_LastKnownStatus']==3]
        df_icu_rec = df_icu[df_icu['BL_LastKnownStatus']==2]
        df_hosp_rec = df_hosp[np.all([df_hosp['BL_LastKnownStatus']==2,
                                      df_hosp['BL_ICU']!='1']
                                     , axis=0)]


        num_hosp = len(df_hosp.values[:,0])
        num_icu = len(df_icu.values[:,0])
        num_dead = len(df_dead.values[:,0])

        num_rec_hosp = len(df_hosp_rec.values[:,0])
        num_rec_icu = len(df_icu_rec.values[:,0])

        if num_hosp > 0:
            mu_HU = num_icu/num_hosp
            mu_HU_dev = z*np.sqrt(mu_HU*(1-mu_HU)/num_hosp)
            mu_HU_dev_list.append(mu_HU_dev)
            mu_HU_list.append(mu_HU)
        else: 
            mu_HU = 'No Entries for this Age Group'
            mu_HU_list.append(0)

        if num_icu > 0:
            mu_UD = num_dead/num_icu
            mu_UD_dev = z*np.sqrt(mu_UD*(1-mu_UD)/num_icu)
            mu_UD_dev_list.append(mu_UD_dev)            
            mu_UD_list.append(mu_UD)
        else:
            mu_UD = 'No Entries for this Age Group'
            mu_UD_list.append(0)

        print('Parameters for Ages', age+ ':')
        print('ICUs per Hosps (', num_hosp,'):', mu_HU, ' 95%-CI: [', mu_HU-mu_HU_dev, ',',mu_HU+mu_HU_dev,']')
        print('Deaths per ICUs (', num_icu,'):', mu_UD, ' 95%-CI: [', mu_UD-mu_UD_dev, ',',mu_UD+mu_UD_dev,']')
        print('')


    if len(mu_UD_list) > 1:
        fig, ax = plt.subplots(1,1,figsize=(9,6))
        ax.plot(range(len(ages)), mu_HU_list)
        ax.plot(range(len(ages)), [x-y for (x,y) in zip(mu_HU_list,mu_HU_dev_list)], 'k--')        
        ax.plot(range(len(ages)), [x+y for (x,y) in zip(mu_HU_list,mu_HU_dev_list)], 'k--') 
        ax.set_ylim(0,1)
        ax.set_xticks(range(len(ages)))
        ax.set_xticklabels(ages)
        ax.set_title('Probability of ICU during hospital stay',fontsize=18)
        ax.set_xlabel('Age Group', fontsize=18)
        ax.set_ylabel('Probability', fontsize=18)
        plt.show()
        
        fig, ax = plt.subplots(1,1,figsize=(9,6))
        ax.plot(range(len(ages)), mu_UD_list)
        ax.plot(range(len(ages)), [x-y for (x,y) in zip(mu_UD_list,mu_UD_dev_list)], 'k--')        
        ax.plot(range(len(ages)), [x+y for (x,y) in zip(mu_UD_list,mu_UD_dev_list)], 'k--')
        ax.set_ylim(0,1)
        ax.set_xticks(range(len(ages)))
        ax.set_xticklabels(ages)
        ax.set_title('Probability of death during ICU',fontsize=18)
        ax.set_xlabel('Age Group', fontsize=18)
        ax.set_ylabel('Probability', fontsize=18)
        plt.show()
        
    return mu_HU_list, mu_UD_list

In [ ]:
# makes unique and inserts zeros into count lists
def create_arrays_to_plot(repeated_list):
    unique, counts = np.unique(repeated_list, return_counts=True)
    
    if(len(unique)>0):   
        xvals = np.arange(max(unique)+1, dtype = int)
        yvals = np.zeros(max(unique)+1, dtype=int)

        for i in range(min(unique),max(unique)+1):
            yvals[unique] = counts

        if(yvals[0]==0):
            xvals = np.delete(xvals, 0) # account for distributions defined for x>0
            yvals = np.delete(yvals, 0)
            
    else:
        xvals = []
        yvals = []
    return (xvals,yvals)

In [ ]:
# fits distribution to values given and checks via Pearsons Chi^2 test
# https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test
# n = the number of cells in the table
# The reduction in the degrees of freedom is calculated as p=s+1, 
# where s is the number of co-variates used in fitting the distribution (normal=2 (mu, sigma), lognormal=3...)
# returns argument 1&2: (x,y) to plot data,
#         argument 3&4: (x,y) to plot best fit, 
#         argument 5&6: distribution name and its parameters (shape (if existent), location and scale),
#         argument 7: SSE, chi2 score and pvalue in one row
def fit_distribution(repeated_list, plot_fit=False):      
    
    xvals, yvals = create_arrays_to_plot(repeated_list)
    yvals_scaled = 1/sum(yvals)*yvals
    
    x = np.linspace(int(min(xvals)),max(repeated_list)+1,200)     
    
    params_list = []
    
    #https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions
    dist_names = ['gamma', 'weibull_min', 'lognorm', 'norm', 'expon']
    error_list = np.zeros([len(dist_names),3])

    i=0
    for dist_name in dist_names:
        # get name of distribution
        dist_name_scipy = getattr(scipy.stats, dist_name)

        # fit dist to data
        params = dist_name_scipy.fit(repeated_list)
        
        # separate parts of parameters
        shape = params[:-2] # shape parameter for e.g. gamma, lognorm and weibull distributions
                            # parameter not available for normal and exponential
        loc = params[-2] # location parameter
        scale = params[-1] # scale parameter
        
        params_list.append(params)        
        
        y_fitted = dist_name_scipy.pdf(xvals, *shape, loc=loc, scale=scale)
        sse = np.power(yvals_scaled - y_fitted, 2.0) # compute squared errors
        
        # compute chi2 and p-value
        chi2 = scipy.stats.chisquare(yvals_scaled, f_exp=y_fitted) 
        sse = np.sum(sse) # compute sum 
        
        error_list[i,0] = sse
        error_list[i,1] = chi2[0]
        error_list[i,2] = chi2[1]
        
        i += 1
        
        
    best_fit_dist_ind = np.argsort(error_list[:,1], axis=0)
    
    for i in range(0,1):
        # separate parts of parameters
        shape = params_list[best_fit_dist_ind[i]][:-2] # shape parameter for e.g. gamma, lognorm and weibull distributions
                            # parameter not available for normal and exponential
        loc = params_list[best_fit_dist_ind[i]][-2] # location parameter
        scale = params_list[best_fit_dist_ind[i]][-1] # scale parameter

        dist_name_scipy = getattr(scipy.stats, dist_names[best_fit_dist_ind[i]])

        y_fitted_plot = dist_name_scipy.pdf(x, *shape, loc=loc, scale=scale)
        
        if plot_fit:
            label_plot = str(dist_names[best_fit_dist_ind[i]])+'('+str(np.around(*shape,2))+','+str(np.around(loc,2))+','+str(np.around(scale,2))+')'

            plt.plot(x, y_fitted_plot, label=label_plot)
            plt.xlim(min(repeated_list),max(repeated_list))
        
    
    if plot_fit:
        plt.plot(xvals, yvals_scaled, label='original data')
        plt.legend(loc='upper right')
        plt.show()  
        print('Best fitted distribution: ',dist_names[best_fit_dist_ind[0]])
        print('\tSSE: ',error_list[best_fit_dist_ind[0],0], "\n\tChi2: ",error_list[best_fit_dist_ind[0],1],"\n\tpvalue: ",error_list[best_fit_dist_ind[0],2])
    
    return (xvals,yvals,x,y_fitted_plot,dist_names[best_fit_dist_ind[0]],params_list[best_fit_dist_ind[i]],error_list[best_fit_dist_ind[0],:])

In [ ]:
# plots original data and is distributional fits
def get_data_fits_and_plot(durations_list, plot_str):
    count_list = []
    unique_list = []    
    dist_params_list = []
    fig, ax = plt.subplots(1,1,figsize=(9,6))
    i=0
    for age in range(len(durations_list)):
        (xvals,yvals,x_fitted_plot,y_fitted_plot,dist_name,params,errors_fitted) = fit_distribution(durations_list[age], False)
        dist_params_list.append([dist_name,params,errors_fitted])
        
        count_list.append(yvals)
        unique_list.append(xvals)
        ax.plot(xvals, yvals/sum(yvals), '-', color=colors[i])     
        ax.plot(x_fitted_plot, y_fitted_plot, '--', color=colors[i]) 
        i+=1
                
    labels = []
    for i in range(len(durations_list)):
        labels.append('Orig. data, age: '+ages[i])
        labels.append(dist_params_list[i][0]+'('+str(np.around(*dist_params_list[i][1][:-2],2))+','+str(np.around(dist_params_list[i][1][-2],2))+','+str(np.around(dist_params_list[i][1][-1],2))+')')
        
    ax.legend(labels, fontsize=min(18,18*6/len(durations_list)))
    ax.set_ylim(0)
    ax.set_title(plot_str, fontsize = 18)
    ax.set_xlabel('Time [days]', fontsize=18)
    ax.set_ylabel('Relative number of patients', fontsize=18)
    plt.show()
    
    return (count_list,unique_list)

In [ ]:
# calculates average time spent in hospital before recovering
def calc_time_hosp_rec(df_age_groups, ages):
    print('Hospital to Recovered (R5):')
    durations_list = []
    mean = []
    std = []
    for df_age, age in zip(df_age_groups, ages):
        print('Parameters for Ages', age+ ':')
        df_hosp_rec_dur = df_age['BL_Duration_InpatientstayNewCat'][np.all([df_age['BL_Inpatient']=='1',
                                                                       df_age['BL_LastKnownStatus']==2, 
                                                                       df_age['BL_ICU']!='1'], axis=0)]

        
        duration = read_column(df_hosp_rec_dur, 2, 15)

        print('Mean of Hospital to Recovered (R5):', np.mean(duration))
        print('STD of Hospital to Recovered (R5):', np.std(duration), '\n')
        durations_list.append(duration)
        mean.append(np.mean(duration))
        std.append(np.std(duration))
        

    unique_list, count_list = get_data_fits_and_plot(durations_list,'Time spent in hospital before recovering')
    
    save_df(ages, unique_list, count_list, 'hosp_to_rec_age' + str(len(ages)))
    
    return mean, std

In [ ]:
R5_mean, R5_std = calc_time_hosp_rec([df], ['All'])

In [ ]:
# calculates average time of positive test before hospital admission 
def calc_time_inf_hosp(df_age_groups, ages):
    print('Infected to Hospital (R6):')
    durations_list = []
    mean = []
    std = []
    for df_age, age in zip(df_age_groups, ages):
        print('Parameters for Ages', age+ ':')
        df_admission = df_age['BL_AdmissionComputedNewCat'][df_age['BL_AdmissionComputedNewCat']!='Missing']


        duration = read_column(df_admission, upper_bound=7)

        print('Mean of Infected to Hospital (R6):', np.mean(duration))
        print('STD of Infected to Hospital (R6):', np.std(duration))
        durations_list.append(duration)
        mean.append(np.mean(duration))
        std.append(np.std(duration))
        print()

        
    unique_list, count_list = get_data_fits_and_plot(durations_list,'Time of known infection before admission')
    
    save_df(ages, unique_list, count_list, 'inf_to_hosp' + str(len(ages)))

    
    return mean, std


In [ ]:
# calculates average time of hospital stay before ICU admission
def calc_time_hosp_ICU(df_age_groups, ages):
    print('Hospital to ICU (R7):')
    durations_list = []
    mean = []
    std = []
    for df_age, age in zip(df_age_groups, ages):
        print('Parameters for Ages', age+ ':')
        df_hosp_icu_dead_dur = df_age[['BL_Duration_InpatientstayNewCat', 
                              'BL_Duration_ICUStayNewCat']][np.all([df_age['BL_Inpatient']=='1',
                                                              df_age['BL_LastKnownStatus']==3, 
                                                              df_age['BL_ICU']=='1'], axis=0)]
        hosp_duration = read_column(df_hosp_icu_dead_dur['BL_Duration_InpatientstayNewCat'], 2, 16)
        icu_duration = read_column(df_hosp_icu_dead_dur['BL_Duration_ICUStayNewCat'], 2, 15)

        durations_list.append(np.array(hosp_duration) - np.array(icu_duration))

        icu_mean = np.mean(np.array(hosp_duration) - np.array(icu_duration))
        print('Mean of Hospital to ICU (R7):', np.mean(np.array(hosp_duration) - np.array(icu_duration)))
        print('STD of Hospital to ICU (R7):', np.std(np.array(hosp_duration) - np.array(icu_duration)))
        mean.append(np.mean(np.array(hosp_duration) - np.array(icu_duration)))
        std.append(np.std(np.array(hosp_duration) - np.array(icu_duration)))
        print()

    
    unique_list, count_list = get_data_fits_and_plot(durations_list,'Time of Hospital Stay before ICU')    
    
    save_df(ages, unique_list, count_list, 'hosp_to_icu' + str(len(ages)))
    
    return mean, std

In [ ]:
# calculates average time of ICU stay before recovering
def calc_time_ICU_rec(df_age_groups, ages):
    print('ICU to Recovered (R8):')
    count_list = []
    unique_list = []
    durations_list = []
    dist_params_list = []
    mean = []
    std = []
    for df_age, age in zip(df_age_groups, ages):
        print('Parameters for Ages', age+ ':')
        df_hosp_icu_dead_dur = df_age[['BL_Duration_InpatientstayNewCat', 
                              'BL_Duration_ICUStayNewCat']][np.all([df_age['BL_Inpatient']=='1',
                                                              df_age['BL_LastKnownStatus']==2, 
                                                              df_age['BL_ICU']=='1'], axis=0)]
        icu_duration = read_column(df_hosp_icu_dead_dur['BL_Duration_ICUStayNewCat'], 2, 15)

        durations_list.append(icu_duration)

        print('Mean of ICU to Recovered (R8):', np.mean(icu_duration))
        print('STD of ICU to Recovered (R8):', np.std(icu_duration))
        
        mean.append(np.mean(np.array(icu_duration)))
        std.append(np.std(np.array(icu_duration)))
        print()
    
    unique_list, count_list = get_data_fits_and_plot(durations_list,'Time of ICU Stay before Recovering')     
    
    save_df(ages, unique_list, count_list, 'icu_to_rec' + str(len(ages)))
    
    return mean, std



In [ ]:
# calculates average time of ICU stay before dying
def calc_time_ICU_death(df_age_groups, ages):
    print('ICU to Death (R10):')
    durations_list = []
    mean = []
    std = []
    for df_age, age in zip(df_age_groups, ages):
        print('Parameters for Ages', age+ ':')
        df_hosp_icu_dead_dur = df_age[['BL_Duration_InpatientstayNewCat', 
                              'BL_Duration_ICUStayNewCat']][np.all([df_age['BL_Inpatient']=='1',
                                                              df_age['BL_LastKnownStatus']==3, 
                                                              df_age['BL_ICU']=='1'], axis=0)]
        icu_duration = read_column(df_hosp_icu_dead_dur['BL_Duration_ICUStayNewCat'], 2, 15)

        durations_list.append(np.array(icu_duration))

        print('Mean of ICU to Death (R10):', np.mean(icu_duration))
        print('STD of ICU to Death (R10):', np.std(icu_duration))
        
        mean.append(np.mean(np.array(icu_duration)))
        std.append(np.std(np.array(icu_duration)))
        print()
    
    unique_list, count_list = get_data_fits_and_plot(durations_list,'Time of ICU Stay before Death')     
    
    save_df(ages, unique_list, count_list, 'icu_to_death' + str(len(ages)))
    
    return mean, std

# All Ages

### Theta and Delta

In [ ]:
theta, delta = calc_prob([df], ['All'])

### Hospital to Recovered (R5)

In [ ]:
R5_mean, R5_std = calc_time_hosp_rec([df], ['All'])

### Infected to Hospital (R6)

In [ ]:
R6_mean, R6_std = calc_time_inf_hosp([df], ['All'])

### Hospital to ICU (R7)

In [ ]:
R7_mean, R7_std = calc_time_hosp_ICU([df], ['All'])

### ICU to Recovered (R8)

In [ ]:
R8_mean, R8_std = calc_time_ICU_rec([df], ['All'])

### ICU to Death (R10)

In [ ]:
R10_mean, R10_std = calc_time_ICU_death([df], ['All'])

In [ ]:
params = pd.DataFrame([theta, delta, 
                       R5_mean, R6_mean, R7_mean, R8_mean, R10_mean, 
                       R5_std, R6_std, R7_std, R8_std, R10_std], 
                      columns=['All'], index=['theta', 'delta', 
                       'R5_mean', 'R6_mean', 'R7_mean', 'R8_mean', 'R10_mean', 
                       'R5_std', 'R6_std', 'R7_std', 'R8_std', 'R10_std']).T
display(params)
params.to_csv('params_1age_groups.csv')

# (Almost) RKI age groups

In [ ]:
# Devide DF into Age Groups

df_age_groups = []

# variant 1
#ages = ['0-4', '4-17', '15-35', '35-65', '65-75', '75+']
#df_age_groups.append(df[np.any([df['BL_Age']==1, df['BL_Age']==2], axis=0)]) # Age 0-4
#df_age_groups.append(df[np.any([df['BL_Age']==3, df['BL_Age']==4, df['BL_Age']==5], axis=0)]) # Age 4-17

#variant 2
ages = ['0-17', '15-35', '35-65', '65-75', '75+']
df_age_groups.append(df[np.any([df['BL_Age']==1, df['BL_Age']==2, df['BL_Age']==3, df['BL_Age']==4, df['BL_Age']==5], axis=0)]) # Age 0-17

df_age_groups.append(df[np.any([df['BL_Age']==13, df['BL_Age']==14, df['BL_Age']==6], axis=0)]) # Age 15-25 # Age 25-35

df_age_groups.append(df[np.any([df['BL_Age']==7, df['BL_Age']==8, df['BL_Age']==9], axis=0)]) # Age 35-45 # Age 45-55 # Age 55-65
df_age_groups.append(df[df['BL_Age']==10]) # Age 65-75
df_age_groups.append(df[np.any([df['BL_Age']==11, df['BL_Age']==12], axis=0)]) # Age 75+

for df_age, age in zip(df_age_groups, ages):
    print('Number of Patients in age group', age + ':\t', len(df_age))

### Hospitalized per ICU (theta) and Death per ICU (delta)

In [ ]:
mu_HU, mu_DU = calc_prob(df_age_groups, ages)

### Hospital to Recovered (R5)

In [ ]:
R5_mean, R5_std = calc_time_hosp_rec(df_age_groups, ages)

### Infected to Hospital (R6)

In [ ]:
R6_mean, R6_std = calc_time_inf_hosp(df_age_groups, ages)

### Hospital to ICU (R7)

In [ ]:
R7_mean, R7_std = calc_time_hosp_ICU(df_age_groups, ages)

### ICU to Recovered (R8)

In [ ]:
R8_mean, R8_std = calc_time_ICU_rec(df_age_groups, ages)

### ICU to Death (R10)

In [ ]:
R10_mean, R10_std = calc_time_ICU_death(df_age_groups, ages)

In [ ]:
params = pd.DataFrame([theta, delta, 
                       R5_mean, R6_mean, R7_mean, R8_mean, R10_mean, 
                       R5_std, R6_std, R7_std, R8_std, R10_std], 
                      columns=ages, index=['theta', 'delta', 
                       'R5_mean', 'R6_mean', 'R7_mean', 'R8_mean', 'R10_mean', 
                       'R5_std', 'R6_std', 'R7_std', 'R8_std', 'R10_std']).T
display(params)

params.to_csv('params_8age_groups.csv')

# 8 Age Groups

In [ ]:
# Devide DF into Age Groups

df_age_groups = []
ages = ['0-4', '4-17', '15-25', '25-35', '35-45', '45-55', '55-65', '65-75', '75+']

df_age_groups.append(df[np.any([df['BL_Age']==1, df['BL_Age']==2], axis=0)]) # Age 0-4
df_age_groups.append(df[np.any([df['BL_Age']==3, df['BL_Age']==4, df['BL_Age']==5], axis=0)]) # Age 4-17
df_age_groups.append(df[np.any([df['BL_Age']==13, df['BL_Age']==14], axis=0)]) # Age 15-25
df_age_groups.append(df[df['BL_Age']==6]) # Age 25-35
df_age_groups.append(df[df['BL_Age']==7]) # Age 35-45
df_age_groups.append(df[df['BL_Age']==8]) # Age 45-55
df_age_groups.append(df[df['BL_Age']==9]) # Age 55-65
df_age_groups.append(df[df['BL_Age']==10]) # Age 65-75
df_age_groups.append(df[np.any([df['BL_Age']==11, df['BL_Age']==12], axis=0)]) # Age 75+

for df_age, age in zip(df_age_groups, ages):
    print('Number of Patients in age group', age + ':\t', len(df_age))

# 3 Age Groups

In [ ]:
df_age_groups = []
ages = ['0-25', '25-55', '55+']

df_age_groups.append(df[np.any([df['BL_Age']==1, df['BL_Age']==2, 
                                df['BL_Age']==3, df['BL_Age']==4, df['BL_Age']==5, 
                                df['BL_Age']==13, df['BL_Age']==14], axis=0)]) # Age 0-25
df_age_groups.append(df[np.any([df['BL_Age']==6, df['BL_Age']==7, 
                                df['BL_Age']==8], axis=0)]) # Age 25-55
df_age_groups.append(df[np.any([df['BL_Age']==9, df['BL_Age']==10, 
                                df['BL_Age']==11, df['BL_Age']==12], axis=0)]) # Age 55+

for df_age, age in zip(df_age_groups, ages):
    print('Number of Patients in age group', age + ':\t', len(df_age))

In [ ]:
from pylab import *
from scipy import stats as st
 
x = [[19.8815 ],[19.0141 ],[18.1857 ],[17.3943 ],[16.6382 ],[15.9158 ],[15.2254 ],[14.5657 ],[13.9352 ],[13.3325 ],[12.7564 ],[12.2056 ],[11.679 ],[11.1755 ],
[22.6941 ],[10.2338 ],[ 9.79353],[ 9.37249],[ 8.96979],[ 8.58462],[ 8.21619],[ 7.86376],[ 7.52662],[ 7.20409],[ 6.89552],[ 6.6003 ],
[ 6.31784],[ 6.04757],[ 5.78897],[ 5.54151],[ 5.30472],[ 5.07812],[ 4.86127],[ 4.65375],[ 4.45514],[ 4.26506],[ 4.08314],[ 3.90903],[ 3.74238],
[ 3.58288],[ 3.4302 ],[ 3.28407],[ 3.14419],[ 3.01029],[ 2.88212],[ 2.75943],[ 2.64198],[ 2.52955],[ 2.42192],[ 2.31889],[ 2.22026],[ 2.12583],
[ 2.03543],[ 1.94889],[ 1.86604],[ 1.78671],[ 1.71077],[ 1.63807],[ 1.56845],[ 1.50181],[ 1.43801],[ 1.37691],[ 1.31842],[ 1.26242],[ 1.2088 ],
[ 1.15746],[ 1.10832],[ 1.06126],[ 1.01619]]
 
 
x = np.array(x).ravel()
 
 
# testing for lognormality
# the data is lognormal if np.log(data) is normal
pvalx = st.shapiro(np.log(x))[-1]
print("p-value for `accepting` lognormality of x-data = ", pvalx)
print("Ok: the array come from lognormal distribution" if pvalx>0.01  else "Hm...  the array isn't lognormal")
 
 
print('Raw mean value of x-data is: ', np.mean(x)) # Note: median and mean values could significantly differ in case of lognormal distribution
print('Mean value of related normal distribution: ',  np.mean(np.log(x))) 
print('Mapped mean value: ',  np.exp(np.mean(np.log(x)))) 
  
s, loc, scale = st.lognorm.fit(x, floc=0) #x0 is rawdata x-axis
estimated_mu = np.log(scale)
 
print("Estimated mu is almost equal to mapped mean value (above):  ", abs(np.exp(estimated_mu)-np.exp(np.mean(np.log(x)))))

In [ ]:
    severe_probs=np.array([0.00050, 0.00165, 0.00720, 0.02080, 0.03430, 0.07650, 0.13280, (0.20655 + 0.24570)/2]) # Overall probability of developing severe symptoms (derived from Table 1 of https://www.imperial.ac.uk/media/imperial-college/medicine/mrc-gida/2020-03-16-COVID19-Report-9.pdf)
    crit_probs=np.array([0.00003, 0.00008, 0.00036, 0.00104, 0.00216, 0.00933, 0.03639, (0.08923 + 0.17420)/2]) # Overall probability of developing critical symptoms (derived from Table 1 of https://www.imperial.ac.uk/media/imperial-college/medicine/mrc-gida/2020-03-16-COVID19-Report-9.pdf)
    death_probs=np.array([0.00002, 0.00006, 0.00030, 0.00080, 0.00150, 0.00600, 0.02200, (0.05100 + 0.09300)/2])